# Spark serverless on Colaboratory

Powered by Open Data Studio [spark-serverless](https://github.com/open-datastudio/spark-serverless) project, it installs Apache Spark 3.0.0 in the serverless configuration.

## How it works
In Spark serverless configuration, the Spark driver runs in Colab, but executors are running on the cloud. So you can scale your job without managing the cluster.

## Cloud platform
Currently, the [spark-serverless](https://github.com/open-datastudio/spark-serverless) project runs executors on [staroid](https://staroid.com), the cloud platform for open-source projects. So you'll need a staroid account to try it.

Contribution for other cloud platform support is welcome!

## Get supported

There’s no such thing as a stupid question!
Don't hesitate to create an [issue](https://github.com/open-datastudio/spark-serverless/issues) or join our [Slack channel](https://join.slack.com/t/opendatastudio/shared_invite/zt-fy2dsmb7-E9_UrBAh4UA47lzN5sUHUA).


## Get involved, get funded

[spark-serverless](https://github.com/open-datastudio/spark-serverless) is an MIT licensed **open-source** project, as a part of the [Open data studio](https://open-datastudio.io) project. Bugfixes, optimizations, new features, ... please feel free to create [pull requests](https://github.com/open-datastudio/spark-serverless/pulls).


If you're listed in Contributors page of the Github repository of the project, you'll get funded by [StarRank](https://staroid.com/site/starrank) when user run spark-serverless on staroid.


## FAQ

Q: Can I install spark-serverless in my laptop or in other environment?

A: Yes, It works everywhere. The same installation/configuration code in this notebook supposed to work in almost everywhere.


# Install

## Setup Kubernetes cluster (staroid)

You need to do this only once, unless you remove the cluster.

  - sign in [staroid](https://staroid.com)
  - Kubernetes -> New Kubernetes cluster

## Configure spark serverless

Set following environment variables

  - `STAROID_ACCESS_TOKEN`
  - `STAROID_ACCOUNT`
  - `STAROID_CLUSTER`
  - `SPARK_INSTANCE_NAME`



In [ ]:
import os
# get a staroid api access token from https://staroid.com/settings/accesstokens
os.environ["STAROID_ACCESS_TOKEN"] = "<YOUR_ACCESS_TOKEN>"
os.environ["STAROID_ACCOUNT"] = "GITHUB/<your github account>"
os.environ["STAROID_CLUSTER"] = "<cluster name, created from 'Setup Kubernetes cluster' section>"
os.environ["SPARK_INSTANCE_NAME"] = "spark1" # instance name of spark serverless to create. You can change as you want. [a-z0-9-]

# Configure image -------------------------------------------------------
# Followings are configured for colab. You don't have to change anything here.
# If you'd like to use it in other environment, you can change followings accordingly.
#
# Spark driver and executor should use the same python version
# 'opendatastudio/spark-py' image includes multiple pythons inside.
# /home/spark/.pyenv/versions/3.6.9/bin/python3
# /home/spark/.pyenv/versions/3.7.7/bin/python3
# /home/spark/.pyenv/versions/3.8.1/bin/python3
# set PYSPARK_PYTHON environment variable based on the python version of the driver.
# In case of colab, we use python 3.6.
os.environ["SPARK_IMAGE"] = "opendatastudio/spark-py:v3.0.0-staroid"
os.environ["PYSPARK_PYTHON"] = "/home/spark/.pyenv/versions/3.6.9/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/bin/python"

## Download binaries

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0/spark-3.0.0-bin-hadoop2.7.tgz
!tar -xf spark-3.0.0-bin-hadoop2.7.tgz
!pip install -q findspark
# staroid cli
!curl -o starctl -s -L https://github.com/staroids/starctl/releases/download/v0.0.1/starctl-linux-amd64 && chmod +x starctl
# kubectl cli
!curl -o kubectl -s -L https://storage.googleapis.com/kubernetes-release/release/v1.16.3/bin/linux/amd64/kubectl && chmod +x kubectl
# clone spark-serverless
!git clone https://github.com/open-datastudio/spark-serverless

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop2.7"
os.environ["STARCTL"] = "/content/starctl"
os.environ["KUBECTL"] = "/content/kubectl"



# Start spark-serverless

Starts a Kubernetes namespace and secure tunnel server on the cloud. It may take few seconds to a minute.

After successful start, secure tunnel is created and Kubernetes API server endpoint is available on http://localhost:8001.

In [ ]:
# start spark serverless
!spark-serverless/bin/ske_start.sh -o "$STAROID_ACCOUNT" -c "$STAROID_CLUSTER" -a "$SPARK_INSTANCE_NAME"

## Create spark session

Creating spark session will create initial number of executor configured

In [ ]:
# generate spark-defaults.conf
!spark-serverless/bin/ske_spark_conf.sh -o "$STAROID_ACCOUNT" -c "$STAROID_CLUSTER" -a "$SPARK_INSTANCE_NAME" > $SPARK_HOME/conf/spark-defaults.conf

import findspark
findspark.init()
from pyspark.sql import SparkSession

# This will creates spark executors on Kubernetes clusters.
# Initial run may take few minutes, in case of cluster provision new nodes for executors.
#
# spark.kubernetes.executor.label.pod.staroid.com/instance-type
#
#    Available values
#    - 'standard-2' (2CPU, 8GB mem)
#    - 'standard-4' (4CPU, 16GB mem)
#    - 'standard-8' (8CPU, 32GB mem)
#
#    Update 'spark.executor.cores' and 'spark.executor.memory' accordingly
#
# spark.kubernetes.executor.label.pod.staroid.com/spot
#
#    'true' to locate executors on Spot instance.
#
spark = SparkSession.builder \
  .config("spark.executor.cores", "4") \
  .config("spark.executor.memory", "16g") \
  .config("spark.kubernetes.executor.label.pod.staroid.com/instance-type", "standard-4") \
  .config("spark.kubernetes.executor.label.pod.staroid.com/spot", "true") \
  .config("spark.dynamicAllocation.initialExecutors", "1") \
  .config("spark.dynamicAllocation.minExecutors", "1") \
  .config("spark.dynamicAllocation.maxExecutors", "10") \
  .getOrCreate()

# Use spark!

Initial run may take 1-5 minutes as cluster provision new nodes for spark executors.

In [ ]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

## Open Spark UI

Run following cell to get Spark UI address.

In [ ]:
%%bash
STAROID_NAMESPACE=`/content/kubectl --server localhost:8001 get pods 2>&1 | sed 's/.*system[:][^:]*[:]\([^:]*\).*/\1/g'`
STAROID_SERVICE_DOMAIN=`/content/kubectl --server localhost:8001 -n $STAROID_NAMESPACE get configmap staroid-envs -o yaml | grep STAROID_SERVICE_DOMAIN | sed 's/[^:]*..\(.*\)/\1/g'`
INSTANCE_NAME=${INSTANCE_NAME:-spark-serverless}

echo "https://p4040-spark-ui-$INSTANCE_NAME--$STAROID_SERVICE_DOMAIN"

# Shutdown & Clean up

In [ ]:
# stop spark instance on the cloud. It does not completely destroy. So it keeps Kubernetes resources (such as ConfigMap, Secret, PersistentVolumeClaim).
!spark-serverless/bin/ske_stop.sh -o "$STAROID_ACCOUNT" -c "$STAROID_CLUSTER" -a "$SPARK_INSTANCE_NAME"

## Important

Visit your Kubernetes cluster page in [staroid](https://staroid.com) and make sure the spark instance is stopped or terminated to prevent unintended usage charges.

You can always stop/terminate instance from the staroid management console, if `ske_stop.sh` command above fails.

